In [2]:
%load_ext sql

In [3]:
%sql postgresql://localhost/names

'Connected: @names'

# SQL III: Questions

## Questions
1. _How many `Kate`s are there in California by year?_


In [3]:
%%sql
SELECT 
     year, SUM(freq) AS kates
FROM 
     name_freq
WHERE 

     state='CA' AND name='Kate'
GROUP BY 
     year
ORDER BY 
     year DESC
LIMIT 10;

 * postgresql://localhost/names
10 rows affected.


year,kates
2016,199
2015,181
2014,210
2013,226
2012,210
2011,221
2010,194
2009,250
2008,319
2007,348


2. _Which year had the most `Kate`s born in California?_

In [4]:
%%sql
SELECT 
     year, SUM(freq) AS kates
FROM 
     name_freq
WHERE 
     state='CA' AND name='Kate'
GROUP BY 
     year
ORDER BY 
     kates DESC
LIMIT 5;

 * postgresql://localhost/names
5 rows affected.


year,kates
2006,348
2007,348
2003,336
2004,333
2005,332


3. _What is the most popular boy's name in the South in 2000?_

  Note: `name_freq_region` is a view we created by joining the region to the `name_freq` table.

In [5]:
%%sql
SELECT 
    name, sum(freq)
FROM 
    name_freq_region
WHERE
         region = 'South'
    AND
         year = 2000
    AND
         gender = 'M'
GROUP BY 
     name
ORDER BY 
     sum(freq) DESC
LIMIT 5;


 * postgresql://localhost/names
5 rows affected.


name,sum
Jacob,12364
Michael,10983
Joshua,10817
Christopher,10179
William,9809


4. _What is the most popular girl's name in the South in 2000?_

In [6]:
%%sql
SELECT 
    name, sum(freq)
FROM 
    name_freq_region
WHERE
         region = 'South'
    AND
         year = 2000
    AND
         gender = 'F'
GROUP BY 
     name
ORDER BY 
     sum(freq) DESC
LIMIT 5;


 * postgresql://localhost/names
5 rows affected.


name,sum
Hannah,9172
Emily,8741
Madison,8087
Sarah,6729
Alexis,6698


5. _Which state has the greatest number of different names in 2000?_

In [7]:
%%sql
SELECT 
    state, COUNT(DISTINCT(name))
FROM 
    name_freq
WHERE
     year = 2000
GROUP BY 
     state
ORDER BY 
     COUNT(DISTINCT(name)) DESC
LIMIT 5;


 * postgresql://localhost/names
5 rows affected.


state,count
CA,5999
TX,4837
NY,4131
FL,3314
IL,3097


6. _Which region has the greatest number of different names in 2000?_

In [8]:
%%sql
SELECT 
    region, COUNT(DISTINCT(name))
FROM 
    name_freq_region
WHERE
     year = 2000
GROUP BY 
     region
ORDER BY 
     COUNT(DISTINCT(name)) DESC
LIMIT 5;


 * postgresql://localhost/names
5 rows affected.


region,count
Pacific,6098
South,5807
Mid_Atlantic,4605
Midwest,3834
None,2408


7. _How many children were born in each state between 2000 and 2010? Treat "Between" as inclusive._

In [9]:
%%sql
SELECT 
     state, sum(freq)
FROM 
     name_freq
WHERE
         year >= 2000
     AND
         year <= 2010
GROUP BY 
     state 
ORDER BY 
     state 
LIMIT 10;

 * postgresql://localhost/names
10 rows affected.


state,sum
AK,62441
AL,486967
AR,296943
AZ,825449
CA,5227565
CO,592577
CT,338803
DC,85876
DE,76744
FL,1957057


8. _Which state doesn't have a region associated with it?_
  Hint: you can find a list of distinct states by looking at
  ```sql
  SELECT DISTINCT(state) FROM name_freq;
  ```
  You should use a JOIN to connect this to the states in the region table.

In [4]:
%%sql
WITH all_states AS (
        SELECT DISTINCT(state) FROM name_freq
    ),
    region_states AS (
        SELECT DISTINCT(state) FROM region
    )
SELECT 
    all_states.state
FROM 
    all_states LEFT JOIN region_states
               ON all_states.state = region_states.state
WHERE
    region_states.state IS NULL;


 * postgresql://localhost/names
1 rows affected.


state
MI


9. _Rank the most popular androgynous names in 2000 (i.e. names that were given to both males and females)?_


  Challenge: There are quite a few popular names such as `Emily` that have a bulk of either male or female. Can you modify this query to calculate the absolute % difference between males and females, and then return those with the smallest difference (i.e. the most 'balanced' androgynous names).

In [5]:
%%sql
WITH boys_names AS 
(
     SELECT  name, SUM(freq) AS n_boys
     FROM    name_freq 
     WHERE   year = 2000
        AND  gender = 'M'
     GROUP BY name
),
girls_names AS
(
     SELECT  name, SUM(freq) AS n_girls
     FROM    name_freq 
     WHERE   year = 2000
        AND  gender = 'F'
     GROUP BY name
)
SELECT 
     boys_names.name, 
     boys_names.n_boys, 
     girls_names.n_girls,
     n_boys + n_girls AS n_both,
     (100 * n_boys) /(n_boys + n_girls) AS pct_boys,
     (100 * n_girls)/(n_boys + n_girls) AS pct_girls,
     ABS((100*n_boys)/(n_boys+n_girls) - 
        (100*n_girls)/(n_boys+n_girls)) AS pct_diff
FROM 
     girls_names INNER JOIN boys_names
                ON girls_names.name = boys_names.name

WHERE 
     ABS((100*n_boys)/(n_boys+n_girls) - 
        (100*n_girls)/(n_boys+n_girls)) < 10
ORDER BY 
      n_both DESC
LIMIT 10;

 * postgresql://localhost/names
10 rows affected.


name,n_boys,n_girls,n_both,pct_boys,pct_girls,pct_diff
Peyton,1976,1946,3922,50,49,1
Casey,1815,1497,3312,54,45,9
Skyler,1462,1262,2724,53,46,7
Devyn,268,291,559,47,52,5
Jaiden,275,270,545,50,49,1
Armani,224,271,495,45,54,9
Baby,253,220,473,53,46,7
Infant,209,208,417,50,49,1
Joan,145,126,271,53,46,7
Notnamed,109,108,217,50,49,1


10. _Which state has the highest % of `John`'s in a 2000?_

In [6]:
%%sql
WITH johns AS (
    SELECT  state, sum(freq) as n_johns
    FROM    name_freq
    WHERE   name = 'John'
       AND  year = 2000
    GROUP BY  
        state
), 
male_births AS ( 
    SELECT state, sum(freq) as m_births
    FROM   name_freq
    WHERE  year = 2000
       AND gender = 'M'
    GROUP BY 
          state
)
SELECT 
      johns.state, 
      n_johns, 
      m_births,
      (100 * n_johns / m_births) as pct_johns
FROM 
      johns INNER JOIN male_births 
            ON johns.state = male_births.state
ORDER BY 
      (100 * n_johns / m_births) ASC
LIMIT 10;

 * postgresql://localhost/names
10 rows affected.


state,n_johns,m_births,pct_johns
IN,359,39297,0
ME,53,5506,0
IA,129,16634,0
ID,54,7868,0
CO,281,28415,0
KS,150,16491,0
AZ,229,37892,0
HI,41,5185,0
CA,1659,247756,0
MI,535,59234,0


## Advanced - window functions

1. _What is the most popular girl's name in the South by year?_
This can be done with either a window function, or using a variation on the following trick:
```sql
select type, variety, price
from fruits
where (
   select count(*) from fruits as f
   where f.type = fruits.type and f.price <= fruits.price
) <= 2;
```

In [79]:
%%sql
WITH girls_names_by_year AS (
   SELECT 
        year, name, SUM(freq) AS n_girls
   FROM 
        name_freq_region
   WHERE 
         region = 'South' AND  gender = 'F'
   GROUP BY 
        year, name
)
SELECT year, name, n_girls
FROM (SELECT year, name, n_girls,
      rank() OVER (PARTITION BY year
                  ORDER BY n_girls DESC ) AS pos
      FROM girls_names_by_year
     ) AS ranked_names
WHERE pos < 2
ORDER BY year DESC
LIMIT 10;

 * postgresql://localhost/names
10 rows affected.


year,name,n_girls
2016,Emma,7278
2015,Emma,7710
2014,Emma,7830
2013,Emma,7879
2012,Emma,7760
2011,Isabella,7490
2010,Isabella,8397
2009,Isabella,7954
2008,Madison,7201
2007,Madison,7693


In [4]:
%%sql
WITH girl_count AS  (
    SELECT 
        year, name, SUM(freq) OVER (PARTITION BY year, name) AS n_girls
    FROM 
        name_freq_region
    WHERE 
        region = 'South' AND  gender = 'F'
) 
SELECT DISTINCT(year), name, n_girls
FROM (
   SELECT 
       year, name, n_girls, MAX(n_girls) OVER (PARTITION BY year) AS max_value
   FROM 
       girl_count
) AS ss
WHERE 
    n_girls = max_value
ORDER BY 
    year DESC
LIMIT 10;

 * postgresql://localhost/names
10 rows affected.


year,name,n_girls
2016,Emma,7278
2015,Emma,7710
2014,Emma,7830
2013,Emma,7879
2012,Emma,7760
2011,Isabella,7490
2010,Isabella,8397
2009,Isabella,7954
2008,Madison,7201
2007,Madison,7693


2. For each region, what is the most popular name for boys in 2010?

In [12]:
%%sql
WITH region_boys AS (
    SELECT 
         region, name, SUM(freq) AS n_boys
    FROM 
        name_freq_region
    WHERE 
        year = 2010
       AND 
        gender = 'M'
    GROUP BY 
        region, name
),
region_max AS (
    SELECT 
        region, MAX(n_boys) AS max_boys
    FROM 
        region_boys
    GROUP BY 
        region
)
SELECT region_boys.region, region_boys.name, region_boys.n_boys
FROM 
    region_max LEFT JOIN region_boys
               ON region_max.max_boys = region_boys.n_boys
              AND region_max.region = region_boys.region
LIMIT 10; 
       

 * postgresql://localhost/names
7 rows affected.


region,name,n_boys
Midwest,Jacob,3845
Pacific,Jacob,4123
Mountain,Jacob,1619
Mid_Atlantic,Michael,3099
South,Jacob,8185
New_England,Jacob,840
None,None,None


In [5]:
%%sql
WITH sums_by_name AS (
SELECT region, name, SUM(freq) AS n_boys
     FROM name_freq_region
     WHERE year = 2010 AND gender = 'M'
     GROUP BY region, name
)
SELECT region, name, n_boys
FROM ( 
    SELECT region, name, n_boys, MAX(n_boys) OVER (PARTITION BY region) AS max_boys
    FROM sums_by_name
) AS max_names
WHERE n_boys = max_boys
LIMIT 10
;

 * postgresql://localhost/names
7 rows affected.


region,name,n_boys
Mid_Atlantic,Michael,3099
Midwest,Jacob,3773
Mountain,Jacob,1619
New_England,Jacob,912
Pacific,Jacob,4123
South,Jacob,8185
None,Jacob,677
